In [1]:
import numpy as np

In [2]:
def entropy(n_samples_per_class,n):
    p_i=np.array(n_samples_per_class)/n
    p_i[p_i==0]=1
    return -np.dot(p_i,np.log(p_i))/np.log(p_i.size)

class Node:
    
    def __init__(self,n_samples_per_class,prediction,num_samples):
        self.num_samples = num_samples
        #self.entropy=entropy(n_samples_per_class,num_samples)
        self.num_samples_per_class = n_samples_per_class
        self.prediction = prediction
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None
    
def countLeaves(node):
    if node.left:
        return countLeaves(node.right)+countLeaves(node.left)
    else:
        return 1
        
                
class DecisionTreeClassifier:
   
    def grow_tree(self,X,y):
        n_samples_per_class=[sum(y==c) for c in range(self.n_classes)]
        prediction=np.argmax(n_samples_per_class)
        #print(prediction)
        node=Node(n_samples_per_class,prediction,len(y))
       
        if len(y)>1:
            idx,threshold=self.best_split(X, y)
            #print("{},{}".format(idx,threshold))
            if idx!=None:
                mask=X[:,idx]<threshold
                node.feature_index=idx
                node.threshold=threshold
                
                node.left=self.grow_tree(X[mask],y[mask])
                node.right=self.grow_tree(X[~mask],y[~mask])
        return node
    
    def best_split(self,X,y):
        n=len(y)
        if n<=1:
            #print("Check")
            return None,None
        
        n_samples_parent=[sum(y==c) for c in range(self.n_classes)]
        entropy_min=1
        
        best_idx=None
        best_threshold=None
                        
        for idx in range(self.n_features):
            threshold_split,values_split=zip(*sorted(zip(X[:,idx],y)))
            
            m_left=[0]*self.n_classes
            n_left=0
            m_right=n_samples_parent.copy()
            n_right=n
            
            for k in range(1,n): 
                
                if threshold_split[k-1]==threshold_split[k]:
                    continue
                
                clss=values_split[k-1]    
                m_left[clss]+=1
                n_left+=1
                m_right[clss]-=1
                n_right-=1
                entropy_split=(n_left/n)*entropy(m_left,n_left)+(n_right/n)*entropy(m_right,n_right)
                
                if entropy_split<entropy_min:
                    best_idx=idx
                    entropy_min=entropy_split
                    best_threshold=(threshold_split[k-1]+threshold_split[k])/2
        return best_idx,best_threshold
    
    def fit(self,X,y):
        self.n_classes=len(set(y))
        self.n_features=X.shape[1]
        self.tree=self.grow_tree(X,y)         
            
    def predict(self,sample):
        node=self.tree        
        while node.left:
            if sample[node.feature_index]<node.threshold:
                node=node.left
            else:
                node=node.right
        return node.prediction
    
    def predictX(self,X):
        return [self.predict(sample) for sample in X]    
    